In [0]:
'''
Use all data from UR dataset and Multiple Camera dataset to tune the ensembled model.
'''

import json
import os
import pandas as pd
import csv
from tensorflow import keras
from numpy import array
import matplotlib.pyplot as plt
import sys

In [0]:
# read the video data csv,

ur_path = '/content/drive/My Drive/636project/ur_data.csv'
mc_fall_path = '/content/drive/My Drive/636project/mc_fall_data.csv'
mc_notfall_path = '/content/drive/My Drive/636project/mc_notfall_data.csv'
youtube_fall_path = '/content/drive/My Drive/636project/youtube_fall_data.csv'
youtube_notfall_path = '/content/drive/My Drive/636project/youtube_notfall_data.csv'

def read_data(path):
  video_data = []
  with open(path, mode='r') as csv_file:
    reader = csv.reader(csv_file)
    for row in reader:
      video_data.append(row)
  return video_data

ur_data = read_data(ur_path)
mc_fall_data = read_data(mc_fall_path)
mc_notfall_data = read_data(mc_notfall_path)
youtube_fall_data = read_data(youtube_fall_path)
youtube_notfall_data = read_data(youtube_notfall_path)

# read target label of UR dataset. 

target = []
path1 = '/content/drive/My Drive/636project/target/urfall-cam0-falls.csv'
path2 = '/content/drive/My Drive/636project/target/urfall-cam0-adls.csv'
# in these csv of UR dataset, 
# '-1' means person is not lying, '1' means person is lying on the ground; '0' is temporary pose, when person "is falling"

def input_fall(path):
  with open(path, mode='r') as csv_file:
    reader = csv.reader(csv_file)
    for row in reader:
      record = []
      record.append(row[0])
      record.append(row[1])
      label = row[2]
      if label == '1' or label == '0':
        record.append(1)        
      else:
        record.append(0)
      # each record: [<video_id>, <frame_id>, label], eg: ['fall-17', '22', 0] 
      target.append(record)

# laying in the adl(activities of daily living) video, but it is not fall, so I mark 0 as label
def input_adl(path):
  with open(path, mode='r') as csv_file:
    reader = csv.reader(csv_file)
    for row in reader:
      record = []
      record.append(row[0])
      record.append(row[1])
      record.append(0)
      target.append(record)

input_fall(path1)
print("falls of UR dataset", len(target))

input_adl(path2)

df = pd.DataFrame.from_records(target)
df.to_csv (r'/content/drive/My Drive/636project/target_data.csv', index = False, header=False)

# make dictionary of target of ur dataset
target_csv_path = '/content/drive/My Drive/636project/target_data.csv'
idx = 0
idx_dict = {}

with open(target_csv_path, mode='r') as csv_file:
  reader = csv.reader(csv_file)
  for row in reader:
    # each row: [<video_id>, <frame_id>, label], eg: ['fall-17', '22', 0] 
    idx_dict['.'.join(row[:2])] = idx  # {'fall-17.22' : 112}
    idx += 1


falls of UR dataset 2995


In [0]:
# match body landmark data with label data of ur dataset
bodylandmark = []
label = []
frame_name = []

for landmark in ur_data:
  video_name = landmark[-1].split('_')  # 'fall-01-cam0_000000000004'
  a = video_name[0].split('-')
  video_type = a[0] # 'fall'
  video_id = a[1]   # '01'
  frame_id = str(int(video_name[1])) # '000000000004' become '4'
  try:
    a = '.'.join([video_id, frame_id])
    b = '-'.join([video_type, a])
    label.append(target[idx_dict[b]][-1])
    bl = landmark[:-1]
    # convert string into float
    bl = list(map(float,bl))
    bodylandmark.append(bl)
    frameName = landmark[-1]
    frame_name.append(frameName)
  except:
    continue

print("bodylandmark record sample of ur dataset:" )
print(bodylandmark[0])
print("label of ur dataset: ", len(label))
print("bodylandmark of ur dataset: ", len(bodylandmark))
print(frame_name[0])

# add data of MC dataset to the whole dataset
# For MC dataset, all frames in fall data should be label 1, 
# and all frames in notfall data should be label 0.
for record in mc_fall_data:
  bl = record[:-1]
  bl = list(map(float,bl))
  bodylandmark.append(bl)
  label.append(1)
  frameName = record[-1]
  frame_name.append(frameName)

for record in mc_notfall_data:
  bl = record[:-1]
  bl = list(map(float,bl))
  bodylandmark.append(bl)
  label.append(0)
  frameName = record[-1]
  frame_name.append(frameName)

# add data of youtube dataset to the whole dataset
# For youtube dataset, all frames in fall data should be label 1, 
# and all frames in notfall data should be label 0.
for record in youtube_fall_data:
  bl = record[:-1]
  bl = list(map(float,bl))
  bodylandmark.append(bl)
  label.append(1)
  frameName = record[-1]
  frame_name.append(frameName)

for record in youtube_notfall_data:
  bl = record[:-1]
  bl = list(map(float,bl))
  bodylandmark.append(bl)
  label.append(0)
  frameName = record[-1]
  frame_name.append(frameName)

# normalize data 
for record in bodylandmark:
  for i in range(0, 63, 3):
    record[i] = float(record[i])/ 640
    record[i + 1] = float(record[i + 1]) / 480

print(bodylandmark[0])
print(len(label))
print(len(bodylandmark))

# make bodylandmark and label to be a same data frame
all_data = []

for i in range(len(bodylandmark)):
  all_data.append([label[i]])
  all_data[i] = all_data[i] + bodylandmark[i]

print(all_data[0])
print(len(all_data))
print(len(all_data[0]))


bodylandmark record sample of ur dataset:
[520.972, 128.178, 0.833728, 544.571, 130.782, 0.739747, 540.636, 132.754, 0.726633, 553.086, 154.999, 0.800764, 541.285, 166.155, 0.742877, 549.144, 129.467, 0.745059, 572.703, 155.035, 0.841464, 543.266, 171.369, 0.817947, 572.067, 169.415, 0.607432, 564.874, 166.16, 0.534525, 526.875, 181.191, 0.862423, 554.389, 217.851, 0.612664, 577.962, 170.732, 0.619415, 535.402, 188.398, 0.760066, 560.284, 220.479, 0.65847, 545.882, 232.266, 0.604373, 553.078, 232.915, 0.532739, 566.184, 222.453, 0.655722, 541.939, 229.002, 0.399536, 541.943, 227.687, 0.344383, 560.285, 217.223, 0.153482]
label of ur dataset:  11475
bodylandmark of ur dataset:  11475
fall-16-cam0_000000000016_keypoints
[0.81401875, 0.2670375, 0.833728, 0.8508921875000001, 0.2724625, 0.739747, 0.8447437499999999, 0.2765708333333333, 0.726633, 0.864196875, 0.32291458333333334, 0.800764, 0.8457578124999999, 0.34615625, 0.742877, 0.8580375, 0.2697229166666667, 0.745059, 0.8948484375, 0.3229

In [0]:
# df = pd.DataFrame.from_records(all_data)
test_cnn = []
test_lstm = []
y_test = []
correct = 0
incorrect = 0
cnn_correct = 0
cnn_incorrect = 0
lstm_correct = 0
lstm_incorrect = 0

for record in all_data:
  test_cnn.append(record[1:])
  test_lstm.append(record[1:])
  label = int(record[0])
  y_test.append(label) 


# use model and write the json file for a video
print("loading model and computing probability for each frame...")
cnn_model = keras.models.load_model('/content/drive/My Drive/636project/model_improved_cnn.h5')
lstm_model = keras.models.load_model('/content/drive/My Drive/636project/model_improved_lstm.h5')

# for cnn model, reshape and predict
test_cnn = array(test_cnn)
test_cnn = test_cnn.reshape((len(test_cnn), 3, int(len(test_cnn[0])/3), 1))
probability_cnn = cnn_model.predict_proba(test_cnn)

# for lstm model, reshape and predict
test_lstm = array(test_lstm)
test_lstm = test_lstm.reshape((len(test_lstm), 1, len(test_lstm[0])))
probability_lstm = lstm_model.predict_proba(test_lstm)

# emsemble the prediction of cnn and lstm model
frame_num = 0

# fp: false positive
# fn: false negative
fp = []
fn = []

for i in range(len(probability_cnn)):
    timestamp = frame_num / 30
    probability_fall = float(probability_cnn[i][0]) * 0.9 + float(probability_lstm[i][0]) * 0.1
  # emsembled model
    if probability_fall < 0.5:
      prediction = 0.0
    else:
      prediction = 1.0

    if prediction == float(y_test[i]):
      correct += 1
    else:
      incorrect += 1
      if prediction == 1.0:
        fp.append(i)
      else:
        fn.append(i)

  # cnn model
    if probability_cnn[i][0] < 0.5:
      prediction_cnn = 0.0
    else:
      prediction_cnn = 1.0

    if prediction_cnn == float(y_test[i]):
      cnn_correct += 1
    else:
      cnn_incorrect += 1

  # lstm model
    if probability_lstm[i][0] < 0.5:
      prediction_lstm = 0.0
    else:
      prediction_lstm = 1.0

    if prediction_lstm == float(y_test[i]):
      lstm_correct += 1
    else:
      lstm_incorrect += 1

print('finish prediction')

print('emsembled model')
print(correct)
print(incorrect)
print(correct / (correct + incorrect))

loading model and computing probability for each frame...
finish prediction
emsembled model
32369
1309
0.9611318961933607


In [0]:
print('emsembled model')
print(correct)
print(incorrect)
print(correct / (correct + incorrect))

emsembled model
31531
2147
0.9362491834431973


In [0]:
print('cnn model')
print(cnn_correct)
print(cnn_incorrect)
print(cnn_correct / (cnn_correct + cnn_incorrect))

cnn model
32370
1308
0.9611615891680029


In [0]:
print('lstm model')
print(lstm_correct)
print(lstm_incorrect)
print(lstm_correct / (lstm_correct + lstm_incorrect))

lstm model
31430
2248
0.9332501930043352


In [0]:
frame_name[0]

'fall-16-cam0_000000000016_keypoints'

In [0]:
# get name of fail frames
false_positive_frames = []
false_negative_frames = []

for fail in fp:
  false_positive_frames.append(frame_name[fail])


for fail in fn:
  false_negative_frames.append(frame_name[fail])

print(len(false_positive_frames))
print(len(false_negative_frames))

1185
124


In [0]:
# save names of fail frames
df_false_negative = pd.DataFrame.from_records(false_negative_frames)
df_false_negative.to_csv (r'/content/drive/My Drive/636project/false_negative_frames.csv', index = False, header=False)
df_false_positive = pd.DataFrame.from_records(false_positive_frames)
df_false_positive.to_csv (r'/content/drive/My Drive/636project/false_positive_frames.csv', index = False, header=False)